<h1>Hello,各位！这里是我对马老师的代码谈谈个人的理解，并且加了一些注释，调整了某些代码块的位置，希望能够帮助大家更好的理解。<span class="tocSkip"></span></h1>
注：大家可以先对每一部分的原理保证能够理解，这样看代码就比较容易，不要上来就硬撸代码，“劳命又伤财，身体最重要！”废话不多说，直接上手！

整体代码其实可以分成三部分组成：
1.乔利斯分解与前/回代法求解方程
2.删除一个变量的简易矩阵运算（givens变换，不改变向量的模，只是旋转方向，可以类比“欧拉公式"在复平面上作用在任意向量的结果）和增加一个变量的简易矩阵运算（forupdate）（为逐步回归做准备，因为逐步就是要删除或者增加变量呀~）
3.逐步回归（这个应该是里面最难理解的一部分了啦，需要详细说明一下）：
注：对于不太懂的同学，建议先看看逐步回归的基本思想，就比较好理解啦！
①在代码中，设定有两个bool类型一维数组，MFLAG和flag，其中最关键的就是flag了，因为它当为TRUE表示方程中将该变量剔除，FALSE表示将该变量进行加回。MFLAG则保证每一趟剔除和增加都包含了常数项。
②AB矩阵，存放着当前还有那些变量保留在方程中，如AB=[1,5,8],这说明方程中第1个，第5个，第8个变量仍保留在方程中。而A矩阵则表示在每一轮剔除和添加的循环中，在该轮循环方程中的保留的变量，在每次循环后，会得到更新。
③AICk，表示在剔除第k个变量后，方程中的AIC的值，用来判断是否应该剔除该变量。
④总结一句话，它的基本思想就是：先把所有的变量放在一起进行回归，得到一个AIC的值，然后先剔除一个变量，剔除标准是选择一个AICk最小的变量进行剔除，同时将对应位置的flag的TRUE变为Flase（以保证后面剔除的变量可以加回到方程中）。最终的停止条件则是无论是增加变量还是剔除变量都无法使AIC降低时，停止迭代，跳出循环。

##由于在这里许多细节内容不方便给大家讲解到位，如果还有不太理解，欢迎大家私下和我在交流哈，有什么不足之处，欢迎批评指正。


# <font color=red>
对于回归问题，该方法的应用实际上是：
$$
    \hat \beta = (X^T X)^{-1} X^T Y \\
    (X^T X) \hat \beta = X^T Y
$$
    设$ X^T X = A, \hat \beta = x, X^T Y = b $,
    则本质上就是求解$ Ax = b $<br>
    接下来就是先利用Cholesky分解将$A$分成$LL^T$，再利用前代法和回代法最终得出 $\hat \beta$
</font>

In [6]:
###第一部分，乔利斯分解###
Cholesky = function(Matr)
{
  m = dim(Matr)[1]
  n = dim(Matr)[2]
  if (m!=n) return ('the Matrix_dim is wrong!')
  L=matrix(0,m,m)
  for(i in 1:m)
  {
    L[i,i]=sqrt(Matr[i,i])
    if(i<m)
    {
      L[(i+1):m,i]=Matr[(i+1):m,i]/L[i,i]
      TL=L[(i+1):m,i]
      TM=outer(TL,TL)
      Matr[(i+1):m,(i+1):m]=Matr[(i+1):m,(i+1):m]-TM
    }
  }
  L
}

In [5]:
##前代法
prefoward=function(L,B)
{
  m=dim(L)[1]
  n=dim(L)[2]
  if(m!=length(B)) return('Wrong dimensions of matrix L or vector b! Please check')
  if(m!=n) return ('the dim of Matrix is Wrong')
  x = rep(0,m)
  for (i in 1:m){
    x[i]=B[i]/L[i,i]
    if(i<m){
      B[(i+1):m]= B[(i+1):m]-x[i]*L[(i+1):m,i]
    }
  }
  x
}

In [4]:
##回代法
backfoward = function(L,B)
{
  m=dim(L)[1]
  n=dim(L)[2]
  if(n!=length(B)) return('Wrong dimensions of matrix L or vector b! Please check')
  if(m!=n) return ('the dim of Matrix is Wrong')
  x = rep(0,m)
  for (i in m:1){
    x[i] = B[i]/L[i,i]
    if(i>1){
      B[(i-1):1]=B[(i-1):1]-x[i]*L[(i-1):1,i]
    }
  }
  x
}

In [10]:
####givens变换，实质上就是把向量旋转到坐标轴上，
## mc和ms就是cos和sin，因为是正交变换，不改变向量的模，只是旋转其方向。
givens = function(mx,lmx){
  # lmx=sqrt(sum(mx*mx))
  mc=mx[1]/lmx
  ms=mx[2]/lmx
  matrix(c(mc,ms,-ms,mc),ncol=2)
}
x1 = c(3,4)
lmx = sqrt(sum(x1*x1))
lmx
x1%*%givens(x1,lmx)

[1] 5

5,-4.440892e-16


In [11]:
####mygivens 变换,一次只去掉第k个变量（删除变量）
mgivens = function(L,k){
  p = dim(L)[1]
  if(k>p) return ('Wrong input of k!')
  
  mk = k
  LK = L[-k,]
  while(mk<p){
    mx = LK[mk,mk:(mk+1)]
    lmx = sqrt(sum(mx*mx))
    LK[mk,mk:(mk+1)] = c(lmx,0)
    if(mk<p-1){
      LK[(mk+1):(p-1),mk:(mk+1)] = LK[(mk+1):(p-1),mk:(mk+1)]%*%givens(mx,lmx)
    }
    mk=mk+1
  }
  return (LK[,-p])
}

In [12]:
##原始矩阵来进行验证。
x = matrix(rnorm(60),10)
xtx = t(x)%*%x
L = Cholesky(xtx)
L
for (k in 1:dim(L)){
    print('k=')
    print(k)
    print(mgivens(L,k))
    xtxk = xtx[-k,-k]
    print(Cholesky(xtxk))
}

2.7283048,0.0000000,0.0000000,0.000000,0.0000000,0.00000
-1.2829700,2.3947625,0.0000000,0.000000,0.0000000,0.00000
-0.2241410,1.1997501,3.0251491,0.000000,0.0000000,0.00000
-0.7383328,1.0284736,2.0287090,3.338521,0.0000000,0.00000
-0.7282535,0.2908277,-1.3148446,1.541557,2.5880110,0.00000
-0.7921550,1.4090805,0.8018228,1.623171,0.7369177,1.83359


Warning message in 1:dim(L):
"numerical expression has 2 elements: only the first used"


[1] "k="
[1] 1
          [,1]       [,2]     [,3]      [,4]     [,5]
[1,] 2.7167811  0.0000000 0.000000 0.0000000 0.000000
[2,] 1.1633925  3.0475703 0.000000 0.0000000 0.000000
[3,] 1.2552387  1.9937896 3.363548 0.0000000 0.000000
[4,] 0.6002658 -1.3662666 1.571691 2.5924379 0.000000
[5,] 1.6161494  0.7919477 1.626884 0.7316331 1.836998
          [,1]       [,2]     [,3]      [,4]     [,5]
[1,] 2.7167811  0.0000000 0.000000 0.0000000 0.000000
[2,] 1.1633925  3.0475703 0.000000 0.0000000 0.000000
[3,] 1.2552387  1.9937896 3.363548 0.0000000 0.000000
[4,] 0.6002658 -1.3662666 1.571691 2.5924379 0.000000
[5,] 1.6161494  0.7919477 1.626884 0.7316331 1.836998
[1] "k="
[1] 2
           [,1]      [,2]     [,3]      [,4]     [,5]
[1,]  2.7283048  0.000000 0.000000 0.0000000 0.000000
[2,] -0.2241410  3.254371 0.000000 0.0000000 0.000000
[3,] -0.7383328  2.264972 3.345002 0.0000000 0.000000
[4,] -0.7282535 -1.115018 1.585552 2.6702727 0.000000
[5,] -0.7921550  1.264815 1.683134 0.9386623 1.96326

In [13]:
####增加一个变量的简便算法
forupdate = function(L,xxk,xkxk){
    lk = prefoward(L,xxk)
    lkk = sqrt(xkxk-sum(lk*lk))
    return (as.matrix(rbind(cbind(L,0),c(lk,lkk))))
}
x <- matrix(rnorm(60),10)
xtx <- t(x)%*%x
# xtx
A <- c(3,2,4)  
# xtx[A,A]
L <- Cholesky(xtx[A,A]) 
k <- 5 #加入第五个变量
xxk <- xtx[A,k,drop =T] 
xkxk = xtx[k,k]
forupdate(L,xxk,xkxk)

A = c(3,2,4,k)
Cholesky(xtx[A,A])

2.18112145,0.0000000,0.0000000,0.000000
1.87747299,2.0279450,0.0000000,0.000000
-0.50334721,-1.8163168,1.2312857,0.000000
0.09132665,0.2707168,-0.9251724,2.134681


2.18112145,0.0000000,0.0000000,0.000000
1.87747299,2.0279450,0.0000000,0.000000
-0.50334721,-1.8163168,1.2312857,0.000000
0.09132665,0.2707168,-0.9251724,2.134681


<font color=red>
<br>需要注意：<br>
前代法和回代法没有本质差别，只是A的形态并不相同，但原理是一致的。<br>
这与接下来的减少一个变量和增加一个变量不同。<br>
因为在减少一个变量的情况下，原矩阵包含所有的信息，因此只需要输入原本的L和需要减少的k。<br>
但在增加一个变量的情况下，因为原矩阵不包含新增加变量的信息，因此后面的函数本质上要输入L，x和k这三个信息（或是L, xxk, xkxk）
</font>


# 对于下段代码中的

RSS <- yty - sum(tb*tb)

有如下解释：

mforwardsolve(...)的结果为$L^T \cdot \beta$，其中$L$是$X^T X$经过Cholesky分解后的下三角阵，$L L^T=X^T X$。
\begin{equation}
RSS=(Y-X\beta)^T(Y-X\beta)\\
=Y^T Y-\beta^T X^T Y-Y^T X\beta+\beta^T X^T X \beta
\end{equation}
因为$\beta^T X^T Y$是一个数，所以$\beta^T X^T Y=Y^T X\beta$，所以
\begin{equation}
RSS=Y^T Y-\beta^T X^T Y\\
=Y^T Y-\beta^T X^T X \beta\\
=Y^T Y-\beta^T L L^T \beta\\
=Y^T Y-(L^T \beta)^T(L^T \beta)
\end{equation}

In [18]:
###逐步回归，
##导入数据集
library(ElemStatLearn)
data(prostate)
data <- prostate[,-10]
head(data)

lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>
-0.5798185,2.769459,50,-1.386294,0,-1.386294,6,0,-0.4307829
-0.9942523,3.319626,58,-1.386294,0,-1.386294,6,0,-0.1625189
-0.5108256,2.691243,74,-1.386294,0,-1.386294,7,20,-0.1625189
-1.2039728,3.282789,58,-1.386294,0,-1.386294,6,0,-0.1625189
0.7514161,3.432373,62,-1.386294,0,-1.386294,6,0,0.3715636
-1.0498221,3.228826,50,-1.386294,0,-1.386294,6,0,0.7654678


In [17]:
steplm = function(data){
    #构造x,xn,y
    n = dim(data)[1]
    p = dim(data)[2]-1  
    xn = c('inter',names(data)[1:p])
#     return (xn)
    x = as.matrix(cbind(1,data[,1:p]))
    y = data[,p+1]
  
    #回归参数
    xtx=t(x)%*%x
    xty=drop(t(x)%*%y)
    yty=sum(y*y)
    
    #求解方程，得到解b
    L = Cholesky(xtx)
    tb = prefoward(L,xty)
    b = backfoward(t(L),tb) 
    
    #计算RSS和AIC
    RSS = yty - sum(tb*tb)
    AICF = n*log(RSS/n) + 2*(p+1)
    
    #初始化
    A = 1:(p+1)
    LA = L
    MAIC = AICF
    mAIC = AICF
    #两个序号参数，控制谁进谁出
    flag = rep(TRUE,p+1)
    MFLAG = c(TRUE,rep(FALSE,p))
    hbb = b
    #逐步回归过程
    repeat{
        #添加第几个变量进来
        if (length(A)<p+1){
            B = (1:(p+1))[!flag]
        }else{
            B = NULL
        }
        AB = c(A,B)#目前是那些变量进入回归模型
        bm = matrix(0,p,p+1)
        AICm = rep(0,p)
        
        ff = 1
        for (k in AB){
            #k是第几个变量
            if(MFLAG[k]){
                ff = ff+1
            }else{
                if (flag[k]){
                    #剔除
                    Lk = mgivens(LA,ff)
                    tA = A[-ff]
                    xtyk =xty[tA]
                    ff = ff +1
                }else{
                    xxk = xtx[A,k,drop =T]
                    xkxk =xtx[k,k]
                    LK = forupdate(LA,xxk,xkxk)
                    tA = c(A,k)
                    xtyk = xty[tA]
                }
                #去掉一个或增加一个变量的结果
                tbk = prefoward(Lk,xtyk)
                bk = backfoward(t(Lk),tbk)
                
                bm[k-1,tA] = bk
                
                #计算RSSK
                RSSk = yty - sum(tbk*tbk)
                AICk = n*log(RSSk/n) + 2*length(tA)
                AICm[k-1] = AICk
                #判断最小AICK是哪个
                if(AICk<mAIC){
                    mink = k
                    mtA = tA
                    mAIC = AICk
                    mLA =Lk
                    hb = bm[k-1,]
                }
            }
        }
        if(mAIC<MAIC){
            flag[mink] = !flag[mink]
            A = mtA
            LA = mLA
            MAIC =mAIC
            hhb = hb
        }
        ##如果所有的都大于，则跳出循环。
        if(mAIC >= MAIC) break
        
    }
    re = data.frame(matrix(hhb[c(flag)],nrow = 1))
    names(re) = xn[flag]
    return (re)
}
# steplm(data)
data = data[,c(sample(1:8,8),9)]
steplm(data)
# data

inter,lcavol,lbph,pgg45,lweight,svi,lcp,age
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.4941548,0.569546,0.09735254,0.005324465,0.6144198,0.7523973,-0.1049594,-0.02091347
